In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install git+https://github.com/ssmilin/pyannote-audio.git@develop

  Cloning https://github.com/ssmilin/pyannote-audio.git (to revision develop) to /tmp/pip-req-build-slb8r1tj
  Running command git clone --filter=blob:none --quiet https://github.com/ssmilin/pyannote-audio.git /tmp/pip-req-build-slb8r1tj
  Resolved https://github.com/ssmilin/pyannote-audio.git to commit 04aaaf68eb0dbb252d3d05d92b723e0520e8fe8d
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 6.0 MB/s eta 

In [4]:
!pip install torchaudio

In [5]:
!git clone https://github.com/ssmilin/speach-diarization.git

Cloning into 'speach-diarization'...
fatal: could not read Username for 'https://github.com': No such device or address


In [6]:
import pyannote.audio
import pyannote.core
import pyannote.database
import pyannote.metrics
import pyannote.pipeline
print(pyannote.audio.__version__)
print(pyannote.core.__version__)
print(pyannote.metrics.__version__)
print(pyannote.pipeline.__version__)
print(pyannote.database.__version__)

2.1.1
5.0.0
3.2.1
2.3
5.0.1


In [7]:
%cd /content/drive/MyDrive/Colab Notebooks/pyannote-database-develop/SRM-diarization-setup-main

/content/drive/MyDrive/Colab Notebooks/pyannote-database-develop/SRM-diarization-setup-main


In [11]:
!python ekapa\ tdnn\ embed\ model.py

2023-08-24 13:04:35.328299: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
'SRM.SpeakerDiarization.mini' found in /content/drive/MyDrive/Colab Notebooks/pyannote-database-develop/SRM-diarization-setup-main/pyannote/full_database.yml does not define the 'scope' of speaker labels (file, database, or global). Setting it to 'file'.
'SRM.SpeakerDiarization.complete' found in /content/drive/MyDrive/Colab Notebooks/pyannote-database-develop/SRM-diarization-setup-main/pyannote/full_database.yml does not define the 'scope' of speaker labels (file, database, or global). Setting it to 'file'.
Protocol SRM.SpeakerDiarization.complete does not provide the path to audio files: adding an 'audio' preprocessor for you. See pyannote.database documentation on how to do that yourself.
Protocol SRM.SpeakerDiarization.complete does not precompute the output of torchaudio.info(): adding a 'torchaudio.info' preprocessor for you to speed up dataloaders. See pyan

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyannote.database import registry
from pyannote.audio.pipelines import SpeakerDiarization
from pyannote.metrics.diarization import DiarizationErrorRate
from pyannote.audio import Inference

In [ ]:
from pyannote.audio.models.embedding import ecapa
registry.load_database("pyannote/full_database.yml")
ami = registry.get_protocol('SRM.SpeakerDiarization.complete')

'SRM.SpeakerDiarization.mini' found in /content/drive/MyDrive/Colab Notebooks/pyannote-database-develop/SRM-diarization-setup-main/pyannote/full_database.yml does not define the 'scope' of speaker labels (file, database, or global). Setting it to 'file'.
'SRM.SpeakerDiarization.complete' found in /content/drive/MyDrive/Colab Notebooks/pyannote-database-develop/SRM-diarization-setup-main/pyannote/full_database.yml does not define the 'scope' of speaker labels (file, database, or global). Setting it to 'file'.


In [ ]:
import matplotlib.pyplot as plt
#ecapa = registry.get_protocol('SRM.Protocol.embedded')
from pyannote.audio.tasks import SpeakerEmbedding
ecapa_task = SpeakerEmbedding(ami, duration=2.0)

ValueError: ignored

In [ ]:
from pyannote.audio.models.embedding import EcapaMFCC
ecapa_model = EcapaMFCC(ecapa_task)
segementation_path = '/content/drive/MyDrive/Colab Notebooks/pyannote-database-develop/SRM-diarization-setup-main/lightning_logs/version_87/checkpoints/epoch=0-step=7.ckpt'
pipeline = SpeakerDiarization(embedding=ecapa_model, segmentation=segementation_path)

NameError: ignored

In [ ]:
pipeline.instantiate({ "segmentation": {
        "threshold": 0.5,
        "min_duration_off": 0.0,
    },
    "clustering": {
        "method": "centroid",
        "min_cluster_size": 15,
        "threshold": 1,
    }})

NameError: ignored

In [ ]:
import pytorch_lightning as pl
checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/pyannote-database-develop/SRM-diarization-setup-main/checkpoints'
trainer = pl.Trainer(devices=2, accelerator="cpu", max_epochs=5, default_root_dir=checkpoint_path)
trainer.fit(ecapa_model)

In [ ]:
ecapa_diarization = Inference(embedding_model)

inf_array = []
# centroids = []
# predict with the model
metric = DiarizationErrorRate()
for file in ami.test():
    # apply finetuned pipeline
    print("Expected output")
    print(file["annotation"])
    expected_output = file["annotation"].get_timeline()
    file["finetuned pipeline"] = pipeline(file)
    print("Calculating final output........ ")
    print(file["finetuned pipeline"] )
    diar , centroid = pipeline.apply(file, return_embeddings=True)
    # centroids.append(centroid)
    xvect_probability = xvector_diarization(file)
    inf_array.append(xvect_probability)

In [ ]:
   # evaluate its performance
    metric(file["annotation"], file["finetuned pipeline"], uem=file["annotated"])
    print(f"The custom trained xvector model reaches a Diarization Error Rate (DER) of {100 * abs(metric):.1f}% on {file}" )

In [ ]:
save_directory = '/content/drive/MyDrive/Colab Notebooks/pyannote-database-develop/SRM-diarization-setup-main/outputs/'

for idx, array in enumerate(inf_array):
    filename = f'{save_directory}array_{idx}.txt'
    np.savetxt(filename, array, delimiter=',')
